<h1 style="color:#33918d; font-weight: bold;">Análisis de datos sobre la evolución del anime</h1>

 <h2 style="color:#cc5064;font-weight: bold;">Obtener los datos para el análisis</h2>

 <h4 style="color:#cc5064;">Primero las librerias y el data set a limpiar</h4>

In [34]:
import os
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import time
from libreria_funciones import *
df = pd.read_json('data.json')

NameError: name 'null' is not defined

 <h4 style="color:#33918d;">Exploración inicial y limpieza superficial</h4>

In [15]:
df.head()

,mal_id,titles,type,source,episodes,rating,score,scored_by,rank,popularity,members,favorites,synopsis,studios,genres,themes
0,1550,"[Attack No.1, ]",TV,Manga,104.0,PG-13 - Teens 13 or older,7.07,4117.0,4124.0,6406,9979,72,Kozue is a middleschool girl and enthusiastic ...,Tokyo Movie Shinsha,"[Drama, Sports]",[Team Sports]
1,2406,"[Sazae-san, Mrs. Sazae]",TV,4-koma manga,NaN,G - All Ages,6.14,1869.0,9126.0,6890,8403,38,The main character is a mother named Sazae-san...,Eiken,"[Comedy, Slice of Life]",[]
2,9895,"[The Bathroom, ]",Movie,Original,1.0,R - 17+ (violence & profanity),4.22,1003.0,13223.0,11636,1779,0,A surrealistic short from minimalist cartoonis...,Kuri Jikken Manga Koubou,"[Avant Garde, Comedy]",[]
3,9163,"[Attack No.1 (1970), ]",Movie,Unknown,1.0,PG-13 - Teens 13 or older,6.39,444.0,7795.0,12299,1410,3,"The first recap film of Attack No.1 , it cover...",,"[Drama, Sports]",[Team Sports]
4,7259,"[Kenju Giga, Anthropo-Cynical Farce]",Movie,Original,1.0,PG-13 - Teens 13 or older,4.92,595.0,12800.0,12805,1206,0,"A dog race is interrupted by a ringmaster, who...",,[],"[Parody, Psychological]"


In [16]:
df.shape

(5056, 16)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5056 entries, 0 to 5055
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   mal_id      5056 non-null   int64  
 1   titles      5056 non-null   object 
 2   type        5055 non-null   object 
 3   source      5056 non-null   object 
 4   episodes    4539 non-null   float64
 5   rating      5022 non-null   object 
 6   score       4817 non-null   float64
 7   scored_by   4817 non-null   float64
 8   rank        4786 non-null   float64
 9   popularity  5056 non-null   int64  
 10  members     5056 non-null   int64  
 11  favorites   5056 non-null   int64  
 12  synopsis    5009 non-null   object 
 13  studios     5056 non-null   object 
 14  genres      5056 non-null   object 
 15  themes      5056 non-null   object 
dtypes: float64(4), int64(4), object(8)
memory usage: 632.1+ KB


In [18]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
mal_id,5056.0,13436.020372,16425.207730,1.0,1427.50,4526.00,23347.50,58391.00
episodes,4539.0,18.509363,62.329765,1.0,1.00,12.00,23.00,1787.00
score,4817.0,7.029498,0.931790,2.0,6.46,7.11,7.67,9.38
scored_by,4817.0,122923.990450,274915.307253,105.0,1723.00,17975.00,118269.00,2795232.00
rank,4786.0,4862.653155,4240.655440,1.0,1320.00,3877.50,7454.00,20609.00
popularity,5056.0,5827.900514,5668.755969,1.0,1049.75,3843.00,9501.50,26508.00
members,5056.0,218764.009691,454454.926491,51.0,3564.00,33540.50,231483.75,3943368.00
favorites,5056.0,6901.799248,32039.765112,0.0,5.00,90.00,1257.00,225867.00


In [19]:
df.describe(include='object').T

,count,unique,top,freq
titles,5056,4516,"[Sazae-san, Mrs. Sazae]",126
type,5055,8,TV,2861
source,5056,17,Manga,2147
rating,5022,6,PG-13 - Teens 13 or older,2444
synopsis,5009,4442,The main character is a mother named Sazae-san...,126
studios,5056,339,,512
genres,5056,630,"[Action, Adventure, Fantasy]",251
themes,5056,561,[],1541


In [20]:
# No me deja ver los valores unicos de todas las columnas porque las columnas contienen listas.
for col in df.select_dtypes(include='object').columns:
    if df[col].apply(lambda x: isinstance(x, list)).any():
        print(f"Columna {col} contiene listas")

Columna titles contiene listas
Columna genres contiene listas
Columna themes contiene listas


In [21]:
# Convertir las listas de genres en columnas separadas
# Primero, creamos una nueva DataFrame con las listas convertidas en series.
genres_split = df['genres'].apply(lambda x: pd.Series(x) if isinstance(x, list) else pd.Series([None]*3))
genres_split.columns = ['genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5','genre_6','genre_7']

# Concatenar las nuevas columnas al DataFrame original
df_nuevo = pd.concat([df.drop(columns=['genres']), genres_split], axis=1)

In [22]:
# Calcular la proporción de valores nulos en las columnas de géneros para ver con cuales me quedo.
proporcion_nulos = df_nuevo[['genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7']].isna().sum() / df_nuevo.shape[0]
print(proporcion_nulos)

genre_1    0.048457
genre_2    0.218948
genre_3    0.523932
genre_4    0.825949
genre_5    0.951741
genre_6    0.989715
genre_7    0.999209
dtype: float64


In [23]:
# Me quedo solo con las 2 primeras.
df_nuevo = df_nuevo.drop(['genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7'], axis=1)
df = df_nuevo

In [24]:
#Me quedo solo con el primer theme.
def extraer_theme_antes_coma(themes):
    if isinstance(themes, list) and len(themes) > 0:
        return themes[0].split(',')[0].strip()
    return None

df['themes'] = df['themes'].apply(extraer_theme_antes_coma)

In [25]:
#Saco de la lista los titulos.
df['titles'] = df['titles'].apply(lambda x: x[0] if isinstance(x, list) and x else x)

In [26]:
df.nunique()

mal_id        4516
titles        4516
type             8
source          17
episodes       160
rating           6
score          472
scored_by     3822
rank          3844
popularity    4358
members       4248
favorites     1725
synopsis      4442
studios        339
themes          50
genre_1         21
genre_2         20
dtype: int64

In [27]:
df.columns

Index(['mal_id', 'titles', 'type', 'source', 'episodes', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'synopsis',
       'studios', 'themes', 'genre_1', 'genre_2'],
      dtype='object')

In [28]:
#Al final themes no me ha parecido interesante.
df.rename(columns={'popularity':'audicence_rank'}, inplace=True)
df = df.drop(['favorites','themes'], axis=1)

,mal_id,titles,type,source,episodes,rating,score,scored_by,rank,audicence_rank,members,synopsis,studios,genre_1,genre_2
0,1550,Attack No.1,TV,Manga,104.0,PG-13 - Teens 13 or older,7.07,4117.0,4124.0,6406,9979,Kozue is a middleschool girl and enthusiastic ...,Tokyo Movie Shinsha,Drama,Sports
1,2406,Sazae-san,TV,4-koma manga,NaN,G - All Ages,6.14,1869.0,9126.0,6890,8403,The main character is a mother named Sazae-san...,Eiken,Comedy,Slice of Life
2,9895,The Bathroom,Movie,Original,1.0,R - 17+ (violence & profanity),4.22,1003.0,13223.0,11636,1779,A surrealistic short from minimalist cartoonis...,Kuri Jikken Manga Koubou,Avant Garde,Comedy
3,9163,Attack No.1 (1970),Movie,Unknown,1.0,PG-13 - Teens 13 or older,6.39,444.0,7795.0,12299,1410,"The first recap film of Attack No.1 , it cover...",,Drama,Sports
4,7259,Kenju Giga,Movie,Original,1.0,PG-13 - Teens 13 or older,4.92,595.0,12800.0,12805,1206,"A dog race is interrupted by a ringmaster, who...",,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5051,56230,Jiisan Baasan Wakagaeru,TV,Web manga,11.0,PG-13 - Teens 13 or older,7.07,9411.0,4141.0,2429,77998,Shouzou and Ine Saitou have been happily marri...,Gekkou,Comedy,Romance
5052,56923,Lv2 kara Cheat datta Motoyuusha Kouho no Matta...,TV,Light novel,12.0,PG-13 - Teens 13 or older,7.02,10747.0,4349.0,2460,76187,The Magical Kingdom of Klyrode summons hundred...,J.C.Staff,Adventure,Fantasy
5053,53835,Unnamed Memory,TV,Light novel,12.0,PG-13 - Teens 13 or older,7.25,8563.0,3091.0,2463,76018,"As a young boy, Prince Oscar Lyeth Increatos L...",ENGI,Adventure,Fantasy
5054,52995,Arifureta Shokugyou de Sekai Saikyou Season 3,TV,Light novel,NaN,PG-13 - Teens 13 or older,NaN,NaN,NaN,2482,75338,Third season of Arifureta Shokugyou de Sekai S...,asread.,Action,Adventure


In [29]:
df = df[['mal_id', 'titles', 'type', 'genre_1', 'genre_2', 'source', 'studios', 'episodes', 'rating', 'score',
       'audicence_rank','scored_by', 'rank', 'members', 'favorites',
       'synopsis']]

In [30]:
# Ahora que tengo los datos un poco ordenados y se que me falta, voy a hacer web scraping

 <h2 style="color:#cc5064;font-weight: bold;">Web Sraping</h2>

In [ ]:
def obtener_animes_top(limit=5550, step=50, delay=1):
    animes = []

    for offset in range(0, limit, step):
        url = f"https://myanimelist.net/topanime.php?limit={offset}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        lista = soup.find_all('tr', class_='ranking-list')

        for anime in lista:
            title_tag = anime.find('h3', class_='anime_ranking_h3')
            title = title_tag.text.strip() if title_tag else None

            info_div = anime.find('div', class_='information di-ib mt4')
            fecha_emision = None
            if info_div:
                lines = info_div.text.strip().split('\n')
                if len(lines) >= 2:
                    fecha_emision = lines[1].strip()

            animes.append({
                'title': title,
                'fecha_emision': fecha_emision
            })

        time.sleep(delay)  # Pausa para evitar sobrecargar el servidor

    return pd.DataFrame(animes)

# Llamo a la función
df_fechas = obtener_animes_top()

In [ ]:
# Me quedo solo con el año.
df_fechas['year'] = df_fechas['fecha_emision'].str.extract(r'(\d{4})').astype(int)

In [ ]:
# Le doy el mismo nombre que en el otro data frame para que funcione merge.
df_fechas.rename(columns={'title': 'titles'}, inplace=True)

In [ ]:
df_n = pd.merge(df,df_fechas,on='titles',how='left')

In [ ]:
#Compruebo si me da una buena cantidad de animes despues de 2020.
animes_2020_en_adelante = df_n[df_n['year'] >= 2020]
print(f"Número de animes desde 2020: {len(animes_2020_en_adelante)}")

 <h2 style="color:#cc5064;font-weight: bold;">API</h2>

In [ ]:
def get_popular_animes(limit=5550):
    url = 'https://api.jikan.moe/v4/anime'
    animes = []
    page = 1
    per_page = 25

    while len(animes) < limit:
        response = requests.get(url, params={'page': page, 'limit': per_page})
        print(f"Página {page}, Código: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            if not data.get('data'):
                print("No hay más datos.")
                break

            animes.extend(data['data'])
            print(f"Total recolectado: {len(animes)}")
            page += 1
            time.sleep(1)  # 🛑 Espera obligatoria para evitar rate limit
        elif response.status_code == 429:
            print("⚠️ Rate limit alcanzado. Esperando 10 segundos...")
            time.sleep(10)  # Espera más larga para recuperarse
        else:
            print(f"❌ Error en página {page}: {response.text}")
            break

    return animes

# Ejecutar función para sacar información de la API
popular_animes = get_popular_animes()

In [ ]:
df_state = pd.DataFrame(popular_animes)
df_state.columns

In [ ]:
columns_to_keep = ['mal_id', 'status', 'airing', 'season', 'year']
df_2 = df_state[columns_to_keep]

 <h4 style="color:#33918d;">Juntamos toda la información obtenida</h4>

In [ ]:
df_final = pd.merge(df_n, df_2, on = 'mal_id', how='left')

In [ ]:
df_final.columns

In [ ]:
df_final['year'] = df_final['year_x'].combine_first(df_final['year_y'])

In [ ]:
df_final.drop(['year_x', 'year_y'], axis=1, inplace=True)

In [ ]:
#Parece que tengo que filtrar los animes por año para que me de animes más nuevos, tendre que usar una funcion distinta para llamar a la api.
animes_2020_en_adelante = df_final[df_final['year'] >= 2020]
print(f"Número de animes desde 2020: {len(animes_2020_en_adelante)}")

In [ ]:
df_final.to_csv('anime_data.csv', index=False)

 <h4 style="color:#33918d;">He pasado este df a un csv que esta en la misma carpeta, y que usare para hacer la limpieza y los insights. Hasta aqui la minería para conseguir la informacion que queria.</h4>